<a href="https://colab.research.google.com/github/jmestanza/time_stretching/blob/master/code/vad_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%cd /content
from IPython.display import clear_output
#!git clone https://github.com/jmestanza/time_stretching.git

#imports
#!pip install librosa
!pip install webrtcvad
import webrtcvad
!pip install --upgrade librosa
import librosa 
import scipy
import numpy as np
from IPython.display import Audio

clear_output()

In [0]:
#local modules
!wget https://raw.githubusercontent.com/jmestanza/time_stretching/master/code/detect_pitch.py
!wget https://raw.githubusercontent.com/jmestanza/time_stretching/master/code/modified_psola.py
!wget https://raw.githubusercontent.com/jmestanza/time_stretching/master/code/vad.py

from modified_psola import modified_psola
from detect_pitch import get_fundamental_frequency
from vad import float2pcm, pcm2float, frame_generator, join_vad_frames, get_vad_frames
from vad import Frame

#audios
!wget "https://raw.githubusercontent.com/vicaub/Pitch-Estimation-and-Voicing-Detection/master/data/fda_ue/rl001.wav"
#!wget "https://raw.githubusercontent.com/jmestanza/time_stretching/master/LPC%2C%20PSOLA%20and%20Pitch%20detection%20speed%20up/input/ahh_without_silences_8k_norm.wav"

clear_output()

In [14]:
fs = 48000

#os.chdir("/content/time_stretching/code/input/")

audio_file = "/content/rl001.wav"
test, _ = librosa.load(audio_file, sr=fs, mono=True)
audio_int = float2pcm(test)


# necesito al menos dos periodos de la señal
#200 = (50+25)+50+(50+25)
#L = 50 # muestras de la ventana
#K = L//2
# tamaño total por observacion
# total = 4*L

#quiero tener 20 milisegundos de señal
T = 20*1e-3
samples = int(T * fs) 
L = samples//4
K = L


Audio(data=audio_int, rate=fs)

In [0]:
#voy a unir todo

vad = webrtcvad.Vad()

#divido el audio en frames de 20 milisegundos
frames = frame_generator(20, audio_int, fs)
frames = list(frames)

speech, not_speech, is_frame_speech = get_vad_frames(frames, vad, fs)

final = join_vad_frames(speech, not_speech, is_frame_speech)



a = librosa.lpc(test, 12)
e_ = scipy.signal.lfilter(a,[1], test) # obtenemos la señal error a traves del filtrado

f0,sample_f0 = get_fundamental_frequency(e_,K,L,fs)

print("Frecuencia fundamental : ",f0," ", sample_f0)
speed = 1.0
error_audio = modified_psola(e_,sample_f0, speed)

new_audio = scipy.signal.lfilter([1],a, error_audio)

#os.chdir("../output")
#librosa.output.write_wav("e_"+audio_file+f"_speed_x{speed}.wav", new_audio, fs)

Audio(data=new_audio,rate=fs)



Audio(data=final, rate=fs)

In [13]:
#esto funcionaaaaa

vad = webrtcvad.Vad()

#divido el audio en frames de 20 milisegundos
frames = frame_generator(20, pcm, fs)
frames = list(frames)

speech, not_speech, is_frame_speech = get_vad_frames(frames, vad, fs)

final = join_vad_frames(speech, not_speech, is_frame_speech)




Audio(data=final, rate=fs)

In [0]:
a = librosa.lpc(ahh, 12)
e_ = scipy.signal.lfilter(a,[1], ahh) # obtenemos la señal error a traves del filtrado

f0,sample_f0 = get_fundamental_frequency(e_,K,L,fs)

print("Frecuencia fundamental : ",f0," ", sample_f0)
speed = 2.0
error_audio = modified_psola(e_,sample_f0, speed)

new_audio = scipy.signal.lfilter([1],a, error_audio)

#os.chdir("../output")
#librosa.output.write_wav("e_"+audio_file+f"_speed_x{speed}.wav", new_audio, fs)

Audio(data=new_audio,rate=fs)

